In [72]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [73]:
#import data and view data frame
data = pd.read_csv(
    "../player_info_reduced.csv")

data = pd.DataFrame(data)

data.columns



Index(['Unnamed: 0', 'player_name', '0304_salary_adjusted',
       '0405_salary_adjusted', '0506_salary_adjusted', '0607_salary_adjusted',
       '0708_salary_adjusted', '0809_salary_adjusted', '0910_salary_adjusted',
       '1011_salary_adjusted', '1112_salary_adjusted', '1213_salary_adjusted',
       '1314_salary_adjusted', '1415_salary_adjusted', '1516_salary_adjusted',
       '1617_salary_adjusted', '1718_salary_adjusted', '1819_salary_adjusted',
       '1920_salary_adjusted', '2021_salary_adjusted', '2122_salary_adjusted',
       '2223_salary_adjusted', '2324_salary', '2425_salary_predicted',
       '2526_salary_predicted', '2627_salary_predicted', 'CONTRACT_START',
       'CONTRACT_END', 'AVG_SALARY', 'AGE', 'GP', 'W', 'L', 'MIN', 'PTS',
       'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB',
       'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', '+/-'],
      dtype='object')

In [74]:
# Drop unnecessary columns
data = data.drop(columns=['Unnamed: 0', 'player_name','2526_salary_predicted', '2627_salary_predicted', 'AVG_SALARY',
                         '1920_salary_adjusted','2021_salary_adjusted', '2122_salary_adjusted', '2223_salary_adjusted',
                         '2324_salary', '2425_salary_predicted'])
data

,0304_salary_adjusted,0405_salary_adjusted,0506_salary_adjusted,0607_salary_adjusted,0708_salary_adjusted,0809_salary_adjusted,0910_salary_adjusted,1011_salary_adjusted,1112_salary_adjusted,1213_salary_adjusted,...,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,+/-
0,6482581,6748393,7041080,8511223,18546814,19514263,21677854,19711923,21033142,22654792,...,73,111,454,565,514,249,104,49,143,618
1,6482581,6748393,7041080,8511223,18546814,19514263,21677854,19711923,21033142,22654792,...,71,51,365,416,511,272,109,49,135,541
2,6136492,7499311,13711048,13362472,14577195,15369740,16279088,17128982,17206990,17566261,...,73,206,486,692,32,88,36,80,240,295
3,6136492,7499311,13711048,13362472,14577195,15369740,16279088,17128982,17206990,17566261,...,72,294,570,864,84,105,42,91,169,230
4,5208750,5422257,5657544,6855081,18546814,19514263,21677854,23313418,24309802,25114603,...,84,145,477,622,242,198,95,51,224,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0,0,0,0,0,0,0,0,0,0,...,76,22,135,157,116,50,20,3,92,-133
195,0,0,0,0,0,0,0,0,0,0,...,40,1,6,7,26,13,4,0,13,-41
196,0,0,0,0,0,0,0,0,0,0,...,73,9,52,61,55,22,26,7,42,47
197,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,1,-6


In [75]:
# Create scaler instance
scaler = StandardScaler()

#isolate features
x = data.drop(columns='1819_salary_adjusted').values.astype(float)

# # separate preprocessed data into features and target arrays
# y = data['1819_salary_adjusted']np.reshape(-1,1)
# x = data.drop(columns='1819_salary_adjusted')

# y = np.reshapeX[:,1].reshape(-1,1)

#fit standardscaler
data_scaler_x = scaler.fit(x)

# scale the data
scaled_x = data_scaler_x.transform(x)

In [90]:
#Isolate target values
y = data['1819_salary_adjusted'].values.astype(float)


# split preprocessed data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(scaled_x, y, random_state=8)

In [97]:
# create linear regression model
reg = SGDRegressor(alpha=0.001, penalty= None)
reg.fit(X_train, y_train)


X_test

array([[-0.18699978, -0.25159379, -0.2902111 , ..., -0.61594114,
        -0.5563093 , -1.38204281],
       [-0.18699978, -0.25159379, -0.2902111 , ..., -0.66219483,
        -1.08659793, -0.07430076],
       [-0.18699978, -0.25159379, -0.2902111 , ...,  1.83550459,
        -0.07281084,  1.26866666],
       ...,
       [-0.18699978, -0.25159379, -0.2902111 , ..., -0.01464313,
         1.6740223 ,  0.18548638],
       [-0.18699978, -0.25159379, -0.2902111 , ..., -0.68532168,
         0.58225159, -0.95493514],
       [-0.18699978, -0.25159379, -0.2902111 , ...,  1.25733342,
         0.95657297,  0.49370841]])

In [98]:
# examine the error rate of our models predictions
error = y_test - reg.predict(X_test)
error 

array([ 1.00121938e+06,  4.02004054e+05,  3.21917325e+06,  4.81910371e+05,
        1.09449821e+06, -8.80775480e+05, -1.20549401e+04,  4.50667077e+06,
       -2.25297981e+06, -1.20320922e+06,  1.37406648e+06,  6.97163074e+05,
        4.17334926e+06,  2.11424755e+06,  7.79326151e+05,  5.95221959e+03,
       -1.24314287e+06,  2.18295526e+06, -1.73665999e+06, -3.05120677e+05,
        1.99199119e+05, -6.15329670e+06,  8.93563582e+06, -5.61412579e+06,
        2.13919777e+05,  1.58477168e+06, -3.93715073e+06, -4.20364111e+06,
       -2.63651435e+06, -2.15998679e+06, -1.81389080e+06,  3.46203537e+06,
        3.56472460e+06,  1.51807448e+07, -1.99980779e+06, -2.64989323e+06,
        8.09539610e+06, -6.05769015e+05,  2.50584224e+05, -5.34718362e+05,
        3.11754978e+06, -1.53646524e+06,  1.47993330e+06, -1.50292238e+05,
        3.19864999e+05, -5.07537035e+06,  5.45839875e+06, -7.73944104e+05,
       -8.17168313e+06,  1.74886147e+06])

In [99]:
# Test model
y_pred = reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)  # setting squared=False gives RMSE directly
r2 = r2_score(y_test, y_pred)

#Print Results
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

Mean Absolute Error: 2625892.9714932307
Mean Squared Error: 14951207283855.893
Root Mean Squared Error: 3866679.1027774587
R-squared: 0.8842589840240634


### These results indicate that the model is still off by  ~3.5 million 

In [81]:
# Adjust the data scaling 
# Create scaler instance
scaler2 = MinMaxScaler()

#fit MinMaxScaler
data_scaler2x = scaler.fit(x)

#scale the data
scaled_data2x = data_scaler2x.transform(x)

In [103]:
# REPEAT - separate preprocessed data into features and target arrays using re-scaled data
X_train, X_test, y_train, y_test = train_test_split(scaled_data2x, y, random_state=22)

#examine target array
data["1819_salary_adjusted"].describe()

count    1.990000e+02
mean     1.620615e+07
std      1.120543e+07
min      1.004870e+05
25%      6.415412e+06
50%      1.469861e+07
75%      2.276547e+07
max      4.404544e+07
Name: 1819_salary_adjusted, dtype: float64

In [100]:
# create linear regression model and change learning rate to 0.01
reg = SGDRegressor(alpha=0.01, penalty= None)
reg.fit(X_train, y_train)


reg

SGDRegressor(alpha=0.01, penalty=None)

In [96]:
# Test model
y_pred = reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)  # setting squared=False gives RMSE directly
r2 = r2_score(y_test, y_pred)

#Print Results
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

Mean Absolute Error: 2682842.2299052756
Mean Squared Error: 14801073207485.965
Root Mean Squared Error: 3847216.293306885
R-squared: 0.8854212092679357
